# Llama Stack Agent Workflows

This notebook contains minimal implementations of common agent workflows discussed in the blog post [Building Effective Agent Workflows](https://www.anthropic.com/research/building-effective-agents). Examples are copied from [anthropic-cookbook](https://github.com/anthropics/anthropic-cookbook/tree/main/patterns/agents). 

In [1]:
MODEL_ID = "meta-llama/Llama-3.3-70B-Instruct"

In [36]:
from llama_stack_client import LlamaStackClient
from llama_stack_client.types.agent_create_params import AgentConfig
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.agent import Agent
from rich.pretty import pprint
import json
import uuid
from pydantic import BaseModel
import rich

client = LlamaStackClient(base_url="http://localhost:8321")

base_agent_config = AgentConfig(
    model=MODEL_ID,
    instructions="You are a helpful assistant.",
    sampling_params={
        "strategy": {"type": "top_p", "temperature": 1.0, "top_p": 0.9},
    },
    toolgroups=[],
    tool_config={
        "tool_choice": "auto",
        "tool_prompt_format": "python_list",
    },
    input_shields=[],
    output_shields=[],
    enable_session_persistence=False,
)

## 1. Basic Multi-LLM Workflow

1. **Prompt Chaining**: Decomposes a task into sequential subtasks, where each step builds on previous results. 

2. **Routing**: Dynamically selects specialized LLM paths based on input characteristics. 

3. **Parallelization**: Distributes independent subtasks acorss multiple LLMs for concurrent processing. 

#### 1.1 Prompt Chaining

In [22]:
vanilla_agent_config = AgentConfig({
    **base_agent_config,
    "instructions": """
    You are a helpful assistant capable of structuring data extraction and formatting. 

    You will be given tasks to extract and format data from a performance report. Here is the report:

    Q3 Performance Summary:
    Our customer satisfaction score rose to 92 points this quarter.
    Revenue grew by 45% compared to last year.
    Market share is now at 23% in our primary market.
    Customer churn decreased to 5% from 8%.
    New user acquisition cost is $43 per user.
    Product adoption rate increased to 78%.
    Employee satisfaction is at 87 points.
    Operating margin improved to 34%.
    """,
})

vanilla_agent = Agent(client, vanilla_agent_config)

session_id_1 = vanilla_agent.create_session(session_name=f"vanilla_agent_{uuid.uuid4()}")

prompts = [
    """Extract only the numerical values and their associated metrics from the text.
    Format each as 'value: metric' on a new line.
    Example format:
    92: customer satisfaction
    45%: revenue growth""",

    """Convert all numerical values to percentages where possible.
    If not a percentage or points, convert to decimal (e.g., 92 points -> 92%).
    Keep one number per line.
    Example format:
    92%: customer satisfaction
    45%: revenue growth""",

    """Sort all lines in descending order by numerical value.
    Keep the format 'value: metric' on each line.
    Example:
    92%: customer satisfaction
    87%: employee satisfaction""",

    """Format the sorted data as a markdown table with columns:
    | Metric | Value |
    |:--|--:|
    | Customer Satisfaction | 92% |""",
]

for i, prompt in enumerate(prompts):    
    response = vanilla_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id_1,
        stream=False,
    )
    print("========= Turn: ", i, "=========")
    print(response.output_message.content)
    print("\n")

========= Turn:  0 =========
Here are the extracted numerical values with their associated metrics:


92: customer satisfaction score
45%: revenue growth
23%: market share
5%: customer churn
43: new user acquisition cost
78%: product adoption rate
87: employee satisfaction
34%: operating margin
8%: customer churn (previous)


========= Turn:  1 =========
Here are the numerical values converted to percentages or decimals:


92%: customer satisfaction
45%: revenue growth
23%: market share
5%: customer churn
8%: previous customer churn
78%: product adoption rate
87%: employee satisfaction
34%: operating margin
0.043: new user acquisition cost (in millions, assuming the cost is $43 per user, but it can also be kept as is: 43: new user acquisition cost, depending on context)


========= Turn:  2 =========
Here are the lines sorted in descending order by numerical value:


92%: customer satisfaction
87%: employee satisfaction
78%: product adoption rate
45%: revenue growth
34%: operating marg

#### 1.2 Routing

In [38]:
# 1. Define a couple of specialized agents
billing_agent_config = AgentConfig({
    **base_agent_config,
    "instructions": """You are a billing support specialist. Follow these guidelines:
    1. Always start with "Billing Support Response:"
    2. First acknowledge the specific billing issue
    3. Explain any charges or discrepancies clearly
    4. List concrete next steps with timeline
    5. End with payment options if relevant
    
    Keep responses professional but friendly.
    """,
})

technical_agent_config = AgentConfig({
    **base_agent_config,
    "instructions": """You are a technical support engineer. Follow these guidelines:
    1. Always start with "Technical Support Response:"
    2. List exact steps to resolve the issue
    3. Include system requirements if relevant
    4. Provide workarounds for common problems
    5. End with escalation path if needed
    
    Use clear, numbered steps and technical details.
    """,
})

account_agent_config = AgentConfig({
    **base_agent_config,
    "instructions": """You are an account security specialist. Follow these guidelines:
    1. Always start with "Account Support Response:"
    2. Prioritize account security and verification
    3. Provide clear steps for account recovery/changes
    4. Include security tips and warnings
    5. Set clear expectations for resolution time
    
    Maintain a serious, security-focused tone.
    """,
})

product_agent_config = AgentConfig({
    **base_agent_config,
    "instructions": """You are a product specialist. Follow these guidelines:
    1. Always start with "Product Support Response:"
    2. Focus on feature education and best practices
    3. Include specific examples of usage
    4. Link to relevant documentation sections
    5. Suggest related features that might help
    
    Be educational and encouraging in tone.
    """,
})

specialized_agents = {
    "billing": Agent(client, billing_agent_config),
    "technical": Agent(client, technical_agent_config),
    "account": Agent(client, account_agent_config),
    "product": Agent(client, product_agent_config),
}

# 2. Define a routing agent
class OutputSchema(BaseModel):
    reasoning: str
    support_team: str

routing_agent_config = AgentConfig({
    **base_agent_config,
    "instructions": f"""You are a routing agent. Analyze the user's input and select the most appropriate support team from these options: 

    {list(specialized_agents.keys())}

    Return the name of the support team in JSON format.

    First explain your reasoning, then provide your selection in this JSON format: 
    {{
        "reasoning": "<your explanation>",
        "support_team": "<support team name>"
    }}

    Note the support team name can only be one of the following: {specialized_agents.keys()}
    """,
    "response_format": {
        "type": "json_schema",
        "json_schema": OutputSchema.model_json_schema()
    }
})

routing_agent = Agent(client, routing_agent_config)

# 3. Create a session for all agents
routing_agent_session_id = routing_agent.create_session(session_name=f"routing_agent_{uuid.uuid4()}")
specialized_agents_session_ids = {
    "billing": specialized_agents["billing"].create_session(session_name=f"billing_agent_{uuid.uuid4()}"),
    "technical": specialized_agents["technical"].create_session(session_name=f"technical_agent_{uuid.uuid4()}"),
    "account": specialized_agents["account"].create_session(session_name=f"account_agent_{uuid.uuid4()}"),
    "product": specialized_agents["product"].create_session(session_name=f"product_agent_{uuid.uuid4()}"),
}

# 4. Combine routing with specialized agents
def process_user_query(query):
    # Step 1: Route to the appropriate support team
    routing_response = routing_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": query,
            }
        ],
        session_id=routing_agent_session_id,
        stream=False,
    )
    try:
        routing_result = json.loads(routing_response.output_message.content)
        rich.print(f"🔀 [cyan] Routing Result: {routing_result['reasoning']} [/cyan]")
        rich.print(f"🔀 [cyan] Routing to {routing_result['support_team']}... [/cyan]")

        # Route to the appropriate support team
        return specialized_agents[routing_result["support_team"]].create_turn(
            messages=[
                {"role": "user", "content": query}
            ],
            session_id=specialized_agents_session_ids[routing_result["support_team"]],
            stream=False,
        )
    except json.JSONDecodeError:
        print("Error: Invalid JSON response from routing agent")
        return None


tickets = [
    """Subject: Can't access my account
    Message: Hi, I've been trying to log in for the past hour but keep getting an 'invalid password' error. 
    I'm sure I'm using the right password. Can you help me regain access? This is urgent as I need to 
    submit a report by end of day.
    - John""",
    
    """Subject: Unexpected charge on my card
    Message: Hello, I just noticed a charge of $49.99 on my credit card from your company, but I thought
    I was on the $29.99 plan. Can you explain this charge and adjust it if it's a mistake?
    Thanks,
    Sarah""",
    
    """Subject: How to export data?
    Message: I need to export all my project data to Excel. I've looked through the docs but can't
    figure out how to do a bulk export. Is this possible? If so, could you walk me through the steps?
    Best regards,
    Mike"""
]

for i, ticket in enumerate(tickets):
    print(f"========= Processing ticket {i+1}: =========")
    response = process_user_query(ticket)
    print(response.output_message.content)
    print("\n")

========= Processing ticket 1: =========


🔀  Routing Result: The user is having trouble accessing their account due to an 'invalid password' error, despite 
being certain they are using the correct password. This issue is related to account access and authentication, 
which falls under the responsibility of the account support team. 

🔀  Routing to account... 

Account Support Response:

Dear John,

We take account security and accessibility very seriously. To ensure the integrity of your account, we must follow a thorough verification process. Before we can assist you with regaining access, we need to confirm your identity.

To initiate the account recovery process, please follow these steps:

1. **Verify your account information**: Please reply to this email with your full name, the email address associated with your account, and the last 4 digits of your phone number (if you have one listed on your account).
2. **Password reset**: We will send you a password reset link to the email address associated with your account. This link will allow you to create a new password. Please note that this link will only be valid for 24 hours.
3. **Security questions**: You may be prompted to answer security questions to further verify your identity.

**Important Security Note**: If you are using a public computer or network, please be cautious when acces

🔀  Routing Result: The user is inquiring about an unexpected charge on their credit card, which suggests a 
billing-related issue. They are also requesting an explanation and potential adjustment of the charge, which 
further indicates that the issue is related to payment or billing. 

🔀  Routing to billing... 

Billing Support Response:

I apologize for the unexpected charge on your credit card, Sarah. I understand that you were expecting to be billed $29.99, but instead, you were charged $49.99. I'm here to help you resolve this issue.

After reviewing your account, I found that the $49.99 charge is due to an upgrade to our premium plan, which was accidentally applied to your account during a recent system update. This upgrade includes additional features that are not part of the standard $29.99 plan.

To correct this, I will immediately downgrade your account back to the $29.99 plan, and I will also process a refund of $20.00, which is the difference between the two plans. You can expect to see the refund credited back to your credit card within the next 3-5 business days.

In the meantime, I will also send you a confirmation email with the updated account details and a receipt for the corrected charge. If you have any further questions or concerns, please don't hesitate to reach out to me 

🔀  Routing Result: The user is seeking assistance with a specific feature or functionality of the product, namely 
exporting data to Excel. This type of inquiry is related to understanding and using the product's capabilities, 
which falls under the scope of the product support team or technical support team. Since the issue is more about 
how to use a feature rather than a technical fault, it leans more towards product support. However, given the 
nature of the request, which involves understanding the technical capabilities of the product, it could also be 
argued that it falls under technical support. Between the two, technical support is more appropriate because it 
often deals with the 'how-to' aspects of using the product's features. 

🔀  Routing to technical... 

Technical Support Response:

Exporting data in bulk to Excel is a feature available in our system. To achieve this, follow these steps:

1. **Login to the system**: Ensure you are logged in with the correct credentials and have the necessary permissions to access and export project data.
2. **Navigate to the Project Dashboard**: Click on the "Projects" tab and select the project for which you want to export data.
3. **Access the Data Export Tool**: In the project dashboard, click on the "Tools" menu and select "Data Export" from the dropdown list.
4. **Select Export Options**: In the Data Export tool, choose the data types you want to export (e.g., tasks, issues, users, etc.). You can select all data types or specific ones based on your requirements.
5. **Choose the Export Format**: Select "Excel (.xlsx)" as the export format from the available options.
6. **Configure Export Settings**: You can configure additional settings such as:
	* Date range: Specify a date range for the data to b